In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import geopy as gp
from matplotlib import rcParams

In [11]:
abnb_df = pd.read_csv('./data/clean/abnb_clean.gz', low_memory=False)
#pd.read_csv(item, skipinitialspace=True, usecols=copy_cols1, sep=',', low_memory = False)
abnb_df = abnb_df.drop(columns = 'Unnamed: 0')
#abnb_df['last_scraped'] = pd.to_datetime(abnb_df['last_scraped'], infer_datetime_format = True )
abnb_df['year'] = pd.to_datetime(abnb_df['last_scraped']).dt.to_period('Y')
abnb_df['availability_pct'] = abnb_df['availability_365']/365 *100
abnb_df["borough"] = abnb_df["borough"].astype("str")
abnb_df["borough"] = abnb_df["borough"].str.upper()


FileNotFoundError: [Errno 2] No such file or directory: './data/clean/abnb_clean.gz'

In [ ]:
acs_df = pd.read_csv('../data/clean/acs_estimate.csv', low_memory=False)

In [ ]:
zrent_df = pd.read_csv('../data/clean/zrentals.csv', low_memory=False)

In [ ]:
zrent_df

In [ ]:
year_count = list(abnb_df.groupby(['year'])['id'].size())
years = ['2015','2016','2017','2018','2019','2020','2021','2022']

In [ ]:
abnb_df.keys()

In [ ]:
def listing_clean(data):
    year_count = list(data.groupby(['year'])['id'].size())
    x=len(years)
    print('Number of NYC Airbnb Listings per Year:')
    for i in range(0,x):
        print('\t'+years[i] +':',year_count[i])
#    print('Number of features in dataset:', data.shape[1])
#    print('Avarage price is : $',round(data.price.mean()))
#    print('Number of hosts/suppliers: ', data.host_id.nunique())
#    print('Number of suburbs: ', data.neighbourhood.nunique())
    return

In [ ]:
listing_clean(abnb_df)

In [ ]:
abnb_df.groupby(['room_type']).count().plot(kind='pie', y='id')
#plt.legend(loc = 'lower right')
plt.title('Ratio of Airbnb by Borough')
plt.show()

In [ ]:
abnb_df.groupby(['borough']).count().plot(kind='pie', y='id')
#plt.legend(loc = 'lower right')
plt.title('Ratio of Airbnb by Borough')
plt.show()

In [ ]:
#color display?
abnb_df[abnb_df['year'] == '2017'].groupby(['borough'])['id'].count().to_frame().sort_values('id').T.plot.barh()

In [ ]:
abnb_df[abnb_df['host_id']==107434423]

In [ ]:
# What would be a good way to show how many airbnb listings are owned 
# by the same owner and/or managed by the same propoerty management company

In [ ]:
abnb_df[abnb_df['host_id']==107434423].groupby(['host_name','year'])['id'].count()

In [ ]:
abnb_df[abnb_df['host_id']==305240193].groupby(['host_name','year'])['id'].count()

In [ ]:
abnb_df[abnb_df['host_id']==219517861].groupby(['host_name','year'])['id'].count()
#Sonder is a short term rental management company. I wonder why the values dropped off from 2019 to 2020

In [ ]:
abnb_df[abnb_df['host_id']==3223938].groupby(['host_name','year'])['id'].count()
# Eugene is someone who works for Settle living a short term rental company offering furnished rooms

In [ ]:
host_test = abnb_df.groupby(['host_id','year'])['id'].count().to_frame().reset_index().sort_values(['id','host_id','year'])

In [ ]:
prop_test = host_test[host_test['id']>15].groupby(['host_id','year'])['id'].sum().to_frame().reset_index()
yr_list = list(prop_test['year'].unique())
yr_list.sort()

In [ ]:
#This definitely ain't it

# for year in yr_list:
#     prop_test[prop_test['year'] == year].groupby(['host_id'])['id'].sum().plot(kind='pie',y='id')
#     plt.title(year+ ' Test')
#     plt.show()

In [ ]:
x = len(years)
for i in range(0,x):
    plt.figure(figsize =(16,10))
    abnb_df[abnb_df['year'] == years[i]].groupby(['borough'])['id'].count().sort_values().plot.barh()
    plt.show()

In [ ]:
def get_groupby_count(df, groupby_cols, value):
    data = df.groupby(groupby_cols)[value].count().to_frame().reset_index()
    data = data.rename(columns={value:'count'})
    data["year"] = data["year"].apply(lambda x: x.year)
    return data

In [ ]:
# Create comparison chart for all boros and NYC based on `value` column
def boro_comparison_line(df, value):
    boro_data = get_groupby_count(df, ['year', 'borough'], value)   
    nyc_data = get_groupby_count(df, ['year'], value)
    nyc_data["borough"] = "NYC"
    
    data = pd.concat([boro_data, nyc_data])

    chart = sns.lineplot(x="year", y="count", hue="borough", data=data)
    chart.axes.set_title("Airbnb count for each borough and NYC".format(value))

In [ ]:
#not working
#boro_comparison_line(abnb_df,'id')

In [ ]:
# Generate zipcode count
def create_boro_count_chart(df, count_col):
    return get_groupby_count(df, [count_col, "borough", "year"], "id").reset_index(drop=True)

In [ ]:
zip_boro_count = create_boro_count_chart(abnb_df, "zipcode")
zip_boro_count.head()

In [ ]:
# HELPER FUNCTION
def map_boro_color(boro):
    boro_color_mapper = {"BRONX": "red", "BROOKLYN": "brown", "MANHATTAN": "blue", "QUEENS": "purple", "STATEN ISLAND": "grey"}
    return boro_color_mapper[boro]

In [ ]:
def map_boro_count(data, boro, year):
    data = data[data["borough"]== boro]
    data = data[data["year"] == year]
    print(data)
    color = map_boro_color(boro) 
    order = data.sort_values(by='count').zipcode
    
    rcParams['figure.figsize'] = 11.7,8.27
    chart = sns.barplot(data=data, y="count", x="zipcode", color=color, order=order)
    chart.axes.tick_params(axis='x', rotation=90)
    chart.set_title("{} zipcode count for year {}".format(boro, year))
    return data


In [ ]:
zip_boro_count

In [ ]:
q = map_boro_count(zip_boro_count, "QUEENS", 2021)

In [ ]:
abnb_df['year'] = abnb_df['year'].astype(str)
acs_df['year'] = acs_df['year'].astype(str)

In [ ]:
acs_df.keys()

In [ ]:
def grouping (df1, df2, group_val, stats):
    plt1 = df1[['year','borough','vacant_housing_units']].sort_values('borough').reset_index(drop=True)
    plt2 =abnb_df.groupby(['borough','year'])['id'].size().to_frame().reset_index()

In [ ]:
#x=len(years)
#for i in range(0,x):
#    acs_df[acs_df['year']== year[i]]
plt1 = acs_df[['year','borough','vacant_housing_units']].sort_values('borough').reset_index(drop=True)
plt2 =abnb_df.groupby(['borough','year'])['id'].size().to_frame().reset_index()
pltf = pd.merge(plt1,plt2, on=['borough','year'])
pltf['year']=pltf['year'].astype(int)
pltf = pltf.sort_values(['borough','year'])
pltf = pltf.rename(columns={'id':'abnb_count'})

In [ ]:
boro_list = list(pltf['borough'].unique())

In [ ]:
for boro in boro_list:
    plt.figure(figsize =(7,3))
    pltf[pltf['borough'] == boro].plot.bar(x='year')
    plt.xlabel('Year') 
    plt.ylabel('Count') 
    plt.title(boro+' - Number of Vacant Housing Units vs. Number of Airbnb Listings per borough')
    plt.show()
    
#pltf.plot.bar(x='borough')
#plt.xticks(rotation=0)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.show()

In [ ]:
plt1 = acs_df[['year','borough','rent_median_value']].sort_values('borough').reset_index(drop=True)
plt2 =abnb_df.groupby(['borough','year'])['id'].size().to_frame().reset_index()
pltf = pd.merge(plt1,plt2, on=['borough','year'])
pltf['year']=pltf['year'].astype(int)
pltf = pltf.sort_values(['borough','year'])
pltf = pltf.rename(columns={'id':'abnb_count'})

In [ ]:
for boro in boro_list:
    plt.figure(figsize =(7,3))
    pltf[pltf['borough'] == boro].plot('year',['abnb_count','rent_median_value'], xlabel='Year'
                                       ,ylabel='Rent Value in dollars', secondary_y='abnb_count')
    plt.ylabel('# of Airbnb Listings') 
    plt.title(boro+' - Median Rent Value (USD) vs. Amount of Airbnb Listings per Borough')
    plt.show()

In [ ]:
zrent_df['borough'].sort_values()

In [ ]:
zrent_df['borough'].replace({'STATEN_ISLAND': 'STATEN ISLAND'})

In [ ]:
zrent_df['time'] = pd.to_datetime(zrent_df['date']).dt.to_period('Y').astype(str)
zrent_df['borough'] = zrent_df['borough'].replace({'STATEN_ISLAND': 'STATEN ISLAND'})
abnb_df['time'] = pd.to_datetime(abnb_df['last_scraped']).dt.to_period('Y').astype(str)
zrent_df = zrent_df.rename(columns={'zip_code':'zipcode'})

In [ ]:
zrent_df.sort_values(['zipcode','time']).head(15)

In [ ]:
plt1 = zrent_df.groupby(['time','borough','zipcode'])['mean_rent'].mean().to_frame().reset_index().sort_values(['zipcode','time'])
plt1

In [ ]:
plt1 = zrent_df.groupby(['time','borough','zipcode'])['mean_rent'].mean().to_frame().reset_index()#.sort_values('borough')
plt2 =abnb_df.groupby(['borough','time','zipcode'])['id'].size().to_frame().reset_index()
pltf = pd.merge(plt1,plt2, on=['borough','time','zipcode'])
#pltf['year']=pltf['year'].astype(int)
pltf = pltf.sort_values(['borough','time'])
pltf = pltf.rename(columns={'id':'abnb_count'})

In [ ]:
pltf[pltf['borough']=='BRONX'].head(30)

In [ ]:
for boro in boro_list:
    plt.figure(figsize =(16,10))
    pltf[pltf['borough'] == boro].plot('time',['abnb_count','mean_rent'], xlabel='Year'
                                       ,ylabel='Mean Rent Value in dollars', secondary_y='abnb_count')
    plt.ylabel('# of Airbnb Listings') 
    plt.title(boro)
    plt.show()

In [ ]:
abnb_df

In [ ]:
def get_boro_top_zipcodes(df, boro, year):
    zipcode_count = abnb_df.groupby(["zipcode", "borough", "year"])["id"].count().to_frame().reset_index()
    zipcode_count = zipcode_count.rename(columns={"id": "count"})
    zipcode_sorted = zipcode_count[["zipcode", "borough", "year", "count"]].sort_values(by=["count"], ascending=False)
    zipcode_grouped = zipcode_sorted.groupby(["year"])
    print(zipcode_grouped)

In [ ]:
get_boro_top_zipcodes(abnb_df, "BRONX", 2019)